In [2]:
import pandas as pd
from spacy import displacy
import spacy
pd.set_option('display.max_colwidth', -1)
import datetime 

In [3]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
# Create our list of punctuation marks
punctuations = string.punctuation
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from datetime import datetime,timedelta
import numpy as np
# sklearn packages
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Loading BERT tokenizer...


In [4]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pickle

with open('/home/ubuntu/abasu/data/nagios/cmc.data.final.pickle', 'rb') as data:
    dataset = pickle.load(data)

X_train=dataset['train'][0]
y_train=dataset['train'][1]

X_test=dataset['test'][0]
y_test=dataset['test'][1]

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser', 'tagger'])

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None, *parg, **kwarg):
        return self

    def transform(self, X):
        # returns the input as a string
        return X[self.key]
    
class OneHotonColumns(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None, *parg, **kwarg):
        self.gen_le = LabelEncoder()
        X_train[self.key]= X_train[self.key].astype(str) 
        X_test[self.key]= X_test[self.key].astype(str) 
        df=pd.concat([X_train[[self.key]],X_test[[self.key]]],axis=0,ignore_index=True)
        self.gen_le.fit(df[self.key])
        self.gen_ohe = OneHotEncoder(handle_unknown='ignore')
        gen_labels = self.gen_le.transform(df[self.key])
        df[self.key+"_label"] = gen_labels
        self.gen_ohe.fit(df[[self.key+"_label"]])
        return self

    def transform(self, X):
        gen_labels = self.gen_le.transform(X[self.key])
        X[self.key+"_label"] = gen_labels
        gen_feature_arr = self.gen_ohe.transform(X[[self.key+"_label"]]).toarray()
        gen_feature_labels = list(self.gen_le.classes_)
        gen_features = pd.DataFrame(gen_feature_arr, 
                            columns=gen_feature_labels)
        return gen_features    
  
from keras.preprocessing.sequence import pad_sequences
class BertDataTransformer(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key,max_len=128):
        self.MAX_LEN=max_len
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        input_ids = []
        sentences=X[self.key]
        # For every sentence...
        for sent in sentences:
            encoded_sent = tokenizer.encode(
                                sent,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                
                           )
            encoded_sent=encoded_sent[: min(self.MAX_LEN,len(encoded_sent))]
            input_ids.append(encoded_sent)
        #Pad sentences to their maxlen
        input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN, dtype="long", 
                                  value=0, truncating="post", padding="post")

        print('\nDone.')
        # Create attention masks
        attention_masks = []
        for sent in input_ids:
            att_mask = [int(token_id > 0) for token_id in sent]
            attention_masks.append(att_mask)
            
        # Convert all inputs and labels into torch tensors, the required datatype 
        # for our model.
        data_inputs = torch.tensor(input_ids)
        data_masks = torch.tensor(attention_masks)
        return {'input':data_inputs, 'mask':data_masks}
    



Using TensorFlow backend.


In [11]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
from transformers import get_linear_schedule_with_warmup
import random
import time,datetime
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))
class BertClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self):
        self.model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
        self.model.cuda()
        # Get all of the model's parameters as a list of tuples.
        params = list(self.model.named_parameters())
        print('The BERT model has {:} different named parameters.\n'.format(len(params)))
        print('==== Embedding Layer ====\n')
        for p in params[0:5]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
        print('\n==== First Transformer ====\n')
        for p in params[5:21]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
        print('\n==== Output Layer ====\n')
        for p in params[-4:]:
            print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
            
        self.optimizer = self.configure_optimizers()
        self.epochs=4
        self.batch_size = 32
        if torch.cuda.is_available():    
            self.device = torch.device("cuda")
            torch.cuda.empty_cache()
        else:
            print('No GPU available, using the CPU instead.')
            self.device = torch.device("cpu")
        
    def configure_optimizers(self):
            param_optimizer = list(self.model.named_parameters())
            no_decay = ["bias", "gamma", "beta"]
            optimizer_grouped_parameters = [
                    {
                        "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                        "weight_decay_rate": 0.01
                        },
                    {
                        "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                        "weight_decay_rate": 0.0
                        },
                    ]
            optimizer = AdamW(
                    optimizer_grouped_parameters,
                    lr=2e-5,
                    )
            return optimizer

    def fit(self, X, y):
        # Create the DataLoader for our training set.
        train_labels = torch.tensor(y)
        train_data = TensorDataset(X['input'], X['mask'], train_labels)
        train_sampler = RandomSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=self.batch_size)
        total_steps = len(train_dataloader) * self.epochs
        scheduler = get_linear_schedule_with_warmup(self.optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
        
        seed_val = 42
        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)
        # Store the average loss after each epoch so we can plot them.
        loss_values = []
        self.model.zero_grad()
        # For each epoch...
        for epoch_i in range(0, self.epochs):
            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, self.epochs))
            print('Training...')
            # Measure how long the training epoch takes.
            t0 = time.time()            # Reset the total loss for this epoch.
            total_loss = 0

            # Set our model to training mode (as opposed to evaluation mode)
            self.model.train()
            # For each batch of training data...
            for step, batch in enumerate(train_dataloader):
                # Progress update every 40 batches.
                if step % 40 == 0 and not step == 0:
                    # Calculate elapsed time in minutes.
                    elapsed = format_time(time.time() - t0)
                    # Report progress.
                    print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

                # Put the model into training mode.    
                self.model.train()
                b_input_ids = batch[0].to(self.device)
                b_input_mask = batch[1].to(self.device)
                b_labels = batch[2].to(self.device)

                # Forward pass (evaluate the model on this training batch)
                # `model` is of type: pytorch_pretrained_bert.modeling.BertForSequenceClassification
                outputs = self.model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels)

                loss = outputs[0]
                # Accumulate the loss. `loss` is a Tensor containing a single value; 
                # the `.item()` function just returns the Python value from the tensor.
                total_loss += loss.item()

                # Perform a backward pass to calculate the gradients.
                loss.backward()

                # Clip the norm of the gradients to 1.0.
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)

                # Update parameters and take a step using the computed gradient
                self.optimizer.step()

                # Update the learning rate.
                scheduler.step()

                # Clear out the gradients (by default they accumulate)
                self.model.zero_grad()

            # Calculate the average loss over the training data.
            avg_train_loss = total_loss / len(train_dataloader)            
            loss_values.append(avg_train_loss)

            print("")
            print("  Average training loss: {0:.2f}".format(avg_train_loss))
            print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        return self

    def predict(self, X):

        validation_labels = torch.tensor(np.zeros(len(X['input'])))
        validation_data = TensorDataset(X['input'], X['mask'], validation_labels)
        validation_sampler = SequentialSampler(validation_data)
        validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=32)

        print("")
        print("Running Predict...")
        t0 = time.time()
        # Put model in evaluation mode to evaluate loss on the validation set
        self.model.eval()
        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        res=[]
        # Evaluate data for one epoch
        for batch in validation_dataloader:
            # Add batch to GPU
            batch = tuple(t.to(self.device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Telling the model not to compute or store gradients, saving memory and speeding up validation
            with torch.no_grad():        
                # Forward pass, calculate logit predictions
                # token_type_ids is for the segment ids, but we only have a single sentence here.
                # See https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L258 
                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = outputs[0]
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            res.append((logits,label_ids))
        retres=[]    
        for lgt,y  in res:
            for i,a in enumerate(lgt):
                retres.append((a[0],a[1],y[i]))
        self._predicted_res=retres
        return self._predicted_res

In [7]:
torch_ip = Pipeline([('selector', BertDataTransformer(key='cmp_str')) ])
trainfeat=torch_ip.transform(X_train)


Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (721 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (721 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (719 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi


Done.


In [ ]:
md=BertClassifier()
md.fit(trainfeat,[int(y) for y in y_train])

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [12]:

torch_ip = Pipeline([('selector', BertDataTransformer(key='cmp_str')) ])
test=torch_ip.transform(X_test)
result=md.predict(test)


Done.

Running Predict...


TypeError: append() takes exactly one argument (2 given)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
res=[]
for lgt,y  in output:
    for i,a in enumerate(lgt):
        res.append((a[0],a[1],y[i]))
    

In [ ]:
c=pd.DataFrame(res,columns=['logit1','logit2','act'])
c[c.act>0].describe()

In [ ]:
c['p']=c.apply(lambda x :1/(1+np.exp(x['logit1'])),axis=1)

In [ ]:
c[c.p>0.05].act.sum(),c[c.p>0.05].shape

In [ ]:
analz=pd.DataFrame({"actual":y_test, "xg":[y for x,y in predicted_xg]})#,"logy":[y for x,y in predicted_logy] })
#analz['pred']=( analz['xg'] + analz['logy'] )/2.0     

In [ ]:
# Predicting with a test dataset
analz['final_pred']=analz.apply(lambda x: 1 if x['xg']>0.76 else 0.0,axis=1)
print(" Accuracy:",metrics.accuracy_score(analz['actual'], analz['final_pred']))
print("Precision:",metrics.precision_score(analz['actual'], analz['final_pred']))
print(" Regression Recall:",metrics.recall_score(analz['actual'], analz['final_pred']))            

In [ ]:
incdf=pd.read_csv("/home/ubuntu/Smit/Data/cmcdata/CMCIncidents.csv")
incdf=incdf[ ['Summary','Status','Description','Issue key','Issue Type','Issue id','Created']]
from datetime import datetime
incdf['created_dt']=incdf.apply(lambda x:  datetime.strptime(x['Created'], '%A %d/%m/%Y %H:%M:%S').date(),axis=1)
incdf=incdf[incdf['Summary'].str.contains("Nagios|nagios")]
#testdfAll=pd.read_csv('/home/ubuntu/abasu/data/nagios/test_final.csv')
mgt=incdf.merge(testDF,how='left', left_on='Issue key', right_on = 'Issuekey')
mgt=mgt[mgt.IncidentFlag==1]
#mgt.to_csv("/home/ubuntu/abasu/data/nagios/test_final_incidents_validate.csv",index=False)
#mgt[mgt.Issuekey=='ITSM-285511'][['Summary',"Description","cmp_str"]]